![Panel HighCharts Logo](https://raw.githubusercontent.com/MarcSkovMadsen/panel-highcharts/main/assets/images/panel-highcharts-logo.png)

# 📈 Panel HighStock Reference Guide

The [Panel](https://panel.holoviz.org) `HighStock` pane allows you to use the powerful [HighCharts](https://www.highcharts.com/) [Stock](https://www.highcharts.com/blog/products/stock/) charts from within the comfort of Python 🐍 and Panel ❤️.

## License

The `panel-highcharts` python package and repository is open source and free to use (MIT License), however the **Highcharts js libraries requires a license for commercial use**. For more info see the Highcharts license [FAQs](https://shop.highsoft.com/faq).

## Parameters:

For layout and styling related parameters see the [Panel Customization Guide](https://panel.holoviz.org/user_guide/Customization.html).

* **``object``** (dict): The initial user `configuration` of the `chart`.
* **``object_update``** (dict) Incremental update to the existing `configuration` of the `chart`. 
* **``event``** (dict): Events like `click` and `mouseOver` if subscribed to using the `@` terminology.

## Methods

* **``add_series``**: The method adds a new series to the chart. Takes the `options`, `redraw` and `animation` arguments.

___

# Usage

## Imports

You must import something from `panel_highcharts` before you run `pn.extension('highstock')`

In [ ]:
import requests, json
import panel_highcharts as ph

ph.config.theme("auto")

Additionally you can specify extra Highcharts `js_files` to include. See the full list at [https://code.highcharts.com](https://code.highcharts.com)

In [ ]:
# ph.config.js_files(highcharts_exporting=False, highcharts_export_data=False, highcharts_solid_gauge=True, ...)

In [ ]:
import panel as pn
pn.extension('highstock')

## Configuration

The `HighStock` pane is configured by providing a simple `dict` to the `object` parameter. For examples see the HighCharts Stock [demos](https://www.highcharts.com/demo/stock).

In [ ]:
data = requests.get('https://cdn.jsdelivr.net/gh/highcharts/highcharts@v7.0.0/samples/data/new-intraday.json').json()

In [ ]:
configuration = {
    "title": {"text": "AAPL stock price by minute"},
    "rangeSelector": {
        "buttons": [
            {"type": "hour", "count": 1, "text": "1h"},
            {"type": "day", "count": 1, "text": "1D"},
            {"type": "all", "count": 1, "text": "All"},
        ],
        "selected": 1,
        "inputEnabled": False,
    },
    "series": [
        {"name": "AAPL", "type": "candlestick", "data": data, "tooltip": {"valueDecimals": 2}}
    ],
}

In [ ]:
chart = ph.HighStock(object=configuration, sizing_mode="stretch_both", min_height=600)
chart

## Layout

In [ ]:
settings = pn.WidgetBox(
    pn.Param(
        chart,
        parameters=["height", "width", "sizing_mode", "margin", "object", "object_update", "event", ],
                widgets={"object": pn.widgets.LiteralInput, "object_update": pn.widgets.LiteralInput, "event": pn.widgets.StaticText},
        sizing_mode="fixed", show_name=False, width=250,
    )
)
pn.Row(settings, chart, sizing_mode="stretch_both")

Try changing the `sizing_mode` to `fixed` and the `width` to `400`.

## Updates

You can *update* the chart by providing a partial `configuration` to the `object_update` parameter.

In [ ]:
object_update = {
    "title": {"text": "Panel HighStock: AAPL stock price by minute"},
}
chart.object_update=object_update

Verify that the `title` and `series` was updated in the chart above.

## Events

You can subscribe to chart events using an the `@` notation as shown below. If you add a string like `@name`, then the key-value pair `'channel': 'name'` will be added to the `event` dictionary.

In [ ]:
event_update = {
    "series": [
        {
            "allowPointSelect": "true",
            "point": {
                "events": {
                    "click": "@click;}",
                    "mouseOver": "@mouseOverFun",
                    "select": "@select",
                    "unselect": "@unselect",
                }
            },
            "events": {
                "mouseOut": "@mouseOutFun",
            }
        }
    ]
}
chart.object_update=event_update

Verify that you can trigger the `click`, `mouseOver`, `select`, `unselect` and `mouseOut` events in the chart above and that the relevant `channel` value is used.

## Javascript

You can use Javascript in the configuration via the `function() {...}` notation.

In [ ]:
js_update = {
    'xAxis': {
            'title': {
                'enabled': True,
                'text': 'Date'
            },
            'labels': {
                'formatter': """function(){return this.value/1000 + "s";}"""
            },
        },
}
chart.object_update=js_update

Verify that the x-axis labels now has `km` units appended in the chart above.

# App

Finally we can wrap it up into a nice app template.

In [ ]:
chart.object = configuration = {
    "title": {"text": "AAPL stock price by minute"},
    "rangeSelector": {
        "buttons": [
            {"type": "hour", "count": 1, "text": "1h"},
            {"type": "day", "count": 1, "text": "1D"},
            {"type": "all", "count": 1, "text": "All"},
        ],
        "selected": 1,
        "inputEnabled": False,
    },
    "series": [
        {
            "name": "AAPL", 
            "type": "candlestick", 
            "data": data, 
            "tooltip": {"valueDecimals": 2},
            "allowPointSelect": "true",
            "point": {
                "events": {
                    "click": "@click;}",
                    "mouseOver": "@mouseOverFun",
                    "select": "@select",
                    "unselect": "@unselect",
                }
            },
            "events": {
                "mouseOut": "@mouseOutFun",
            }
        }
    ],
}

In [ ]:
app = pn.template.FastListTemplate(
    site="Panel Highcharts", site_url="./",
    title="HighStock Reference Example", 
    sidebar=[settings], 
    main=[chart]
).servable()

You can serve with `panel serve HighStock.ipynb` and explore the app at http://localhost:5006/HighStock.

Add the `--autoreload` flag to get *hot reloading* when you save the notebook.

![HighStock Reference Guide](https://raw.githubusercontent.com/MarcSkovMadsen/panel-highcharts/main/assets/images/HighStockApp.gif)